In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
from pprint import pprint
import csv
import random
import requests, re, json
from bs4 import BeautifulSoup

In [2]:
def datareader(club_name, season):
    # Read the CSV files
    transfers_df = pd.read_csv('transfers.csv')
    games_df = pd.read_csv('games.csv')
    lineups_df = pd.read_csv('game_lineups.csv')
    
    # Filter transfers for the given club and season
    club_transfers = transfers_df[(transfers_df['to_club_name'] == club_name) & (transfers_df['transfer_season'] == season)].copy()
    
    if club_transfers.empty:
        print(f"No transfers found for {club_name} in {season} season.")
        return pd.DataFrame()
    
    # Get the club_id
    club_id = club_transfers['to_club_id'].iloc[0]
    
    # Filter games for the club, season, and specific competition and round
    season_year = int(season.split('/')[0]) + 2000
    club_games = games_df[(games_df['home_club_id'] == club_id) | (games_df['away_club_id'] == club_id)].copy()
    club_games = club_games[(club_games['season'] == season_year) & (club_games['competition_id'] == 'GB1') & (club_games['round'] == '38. Matchday')]
    
    # Get the club's position
    if not club_games.empty:
        if club_games['home_club_id'].iloc[0] == club_id:
            position = club_games['home_club_position'].iloc[0]
        else:
            position = club_games['away_club_position'].iloc[0]
    else:
        print(f"No matching game found for {club_name} in {season} season.")
        position = np.nan
    
    # Calculate starting percentage
    lineups_for_club = lineups_df[lineups_df['club_id'] == club_id].copy()
    player_appearances = lineups_for_club.groupby('player_id').agg(total_appearances=('type', 'count'),starting_appearances=
                                                                   ('type', lambda x: (x == 'starting_lineup').sum()))
    player_appearances['starting_percentage'] = (player_appearances['starting_appearances'] / player_appearances['total_appearances'] * 100).round(2)
    
    # Create the final dataframe
    result_df = club_transfers[['player_id', 'transfer_fee', 'market_value_in_eur']].copy()
    result_df.loc[:, 'club_id'] = club_id
    result_df.loc[:, 'club_position'] = position
    
    # Merge with starting percentages
    result_df = result_df.merge(
        player_appearances[['starting_percentage']], 
        left_on='player_id', 
        right_index=True, 
        how='left'
    )
    
    # Fill NaN values with 0 for starting_percentage
    result_df.loc[:, 'starting_percentage'] = result_df['starting_percentage'].fillna(0)
    
    return result_df

In [3]:
# Example usage
club_name = "Wolves"
season = "18/19"
df = datareader(club_name, season)
print(df)

       player_id  transfer_fee  market_value_in_eur  club_id  club_position  \
41999     233124           0.0            1000000.0      543            7.0   
42270     361254           0.0            3000000.0      543            7.0   
42987     175446    21000000.0           12000000.0      543            7.0   
43699     180843           0.0             500000.0      543            7.0   
44724     168157           0.0           17000000.0      543            7.0   
44746     525247     4750000.0                  NaN      543            7.0   
44757     204103    20000000.0            8000000.0      543            7.0   
45038     175446           0.0           12000000.0      543            7.0   
45083      29364     5600000.0           10000000.0      543            7.0   
45604      45026    18000000.0           16000000.0      543            7.0   
45743     142310    12000000.0            5000000.0      543            7.0   
45839     206040           0.0           15000000.0 